# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
# Passo 1 : Pegar Cotação do Dólar
# Passo 2 : Pegar Cotação do Euro
# Passo 3 : Pegar Cotação do Ouro
# Passo 4 : Importar a Base de Dados
# Passo 5 : Atualizar a Cotação, o Preço de Compra e Preço de Venda
# Passo 6 : Exportar o Relatório Atualizado

# Instalar o Selenium
# Baixar o WebDriver

!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome("chromedriver.exe")
# Passo 1: Pegar a Cotação do Dólar
# Entrar no site do Google
browser.get("https://google.com/")
# Pesquisar o valor do Dólar
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# Pegar a cotação da página do Google
dolar = browser.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação do Dólar: " ,dolar)

Cotação do Dólar:  5.5152


In [3]:
# Passo 2 : Pegar Cotação do Euro
# Entrar no site do Google
browser.get("https://google.com/")
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
euro = browser.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação do Euro: " ,euro)

Cotação do Euro:  6.382350466


In [4]:
# Passo 2 : Pegar Cotação do Ouro
# Entrar no site do Google
browser.get("https://www.melhorcambio.com/ouro-hoje")
ouro = browser.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
ouro = ouro.replace(",",".")
print("Cotação do Ouro: " ,ouro)

Cotação do Ouro:  311.70


In [5]:
# Passo 4 : Importar a Base de Dados
import pandas as pd

df = pd.read_excel("Produtos.xlsx")
print(df)

         Produtos  Preço Base Original  Moeda  Cotação  Preço Base Reais  \
0    Câmera Canon               999.99  Dólar        5           4999.95   
1   Carro Renault              4500.00   Euro        6          27000.00   
2   Notebook Dell               899.99  Dólar        5           4499.95   
3          IPhone               799.00  Dólar        5           3995.00   
4      Carro Fiat              3000.00   Euro        6          18000.00   
5  Celular Xiaomi               480.48  Dólar        5           2402.40   
6        Joia 20g                20.00   Ouro      350           7000.00   

   Margem  Preço Final  
0    1.40     6999.930  
1    2.00    54000.000  
2    1.70     7649.915  
3    1.70     6791.500  
4    1.90    34200.000  
5    2.00     4804.800  
6    1.15     8050.000  


In [6]:
# Passo 5 : Atualizar a Cotação 
# Atualizar tabela

df.loc[df["Moeda"] == "Dólar", "Cotação"] = float(dolar)
df.loc[df["Moeda"] == "Euro", "Cotação"] = float(euro)
df.loc[df["Moeda"] == "Ouro", "Cotação"] = float(ouro)

In [7]:
# Atualizar o Preço de Compra e Preço de Venda * Cotação
df["Preço Base Reais"] = df["Preço Base Original"] * df["Cotação"]


In [8]:
# Atualizar o Preço de Venda - Preço de Compra * Margem
df["Preço Final"] = df["Preço Base Reais"] * df["Margem"]
print(df)

df["Preço Final"] = df["Preço Final"].map("${:2f}".format)
print(df)

         Produtos  Preço Base Original  Moeda    Cotação  Preço Base Reais  \
0    Câmera Canon               999.99  Dólar    5.51520       5515.144848   
1   Carro Renault              4500.00   Euro    6.38235      28720.577097   
2   Notebook Dell               899.99  Dólar    5.51520       4963.624848   
3          IPhone               799.00  Dólar    5.51520       4406.644800   
4      Carro Fiat              3000.00   Euro    6.38235      19147.051398   
5  Celular Xiaomi               480.48  Dólar    5.51520       2649.943296   
6        Joia 20g                20.00   Ouro  311.70000       6234.000000   

   Margem   Preço Final  
0    1.40   7721.202787  
1    2.00  57441.154194  
2    1.70   8438.162242  
3    1.70   7491.296160  
4    1.90  36379.397656  
5    2.00   5299.886592  
6    1.15   7169.100000  
         Produtos  Preço Base Original  Moeda    Cotação  Preço Base Reais  \
0    Câmera Canon               999.99  Dólar    5.51520       5515.144848   
1   Carro R

In [9]:
# Passo 6 : Exportar o Relatório Atualizado
df.to_excel("Produtos_Atualizados.xlsx", index=False)
browser.quit()